# A4: Best Comments

In this assignment you will be modifying a recursive function that prints a comments and replies on a reddit post. Your goal will be to only show the best comments and replies. It will be up to you to decide what rules you use to decide which comments are the best comments.

## Reddit Praw Setup

In [1]:
# make sure praw library is installed
import praw

In [2]:
%run reddit_keys.py

In [3]:
# Give the praw code your reddit account info so
# it can perform reddit actions
reddit = praw.Reddit(
    username=username, password=password,
    client_id=client_id, client_secret=client_secret,
    user_agent="a custom python script for user /" + str(username)
)

### Helper function to display text in an indented box
(You don't need to worry about how this works)

In [4]:
from IPython.display import HTML, Image, display
import html
def display_indented(text, left_margin=0, color="white"):
    display(
        HTML(
            "<pre style='border:solid 1px;padding:3px;margin-left:"+str(left_margin)+"px;background-color:"+color+"'>" + 
            html.escape(text) + 
            "</pre>"
        )
    )

## Code to print a post with all comments and replies
We are providing these function that recursively prints all comments and replies, but depends on whether a `should_display` function returns True or False to decide if it actually displays a comment. (Note: if a `should_display` comes back false for a comment, the comment wont be displayed, nor will any replies to it)

The `print_post_and_replies` is a function that takes a postId (instructions on where to get one later), prints information about that post, and then uses the `print_comment_and_replies` function to print out all comments and replies.

In [5]:
def print_post_and_replies(postId, show_hidden=False):
    submission = reddit.submission(postId)
    
    print("Comments and replies for post from /"+ submission.subreddit.display_name + ":" )
    display(HTML('<a href="'+"https://www.reddit.com/" + submission.permalink +'">'+submission.title+'</a>'))
    
    submission.comment_sort = "old"
    submission.comments.replace_more()
    comments = submission.comments
    
    for comment in comments:
        print_comment_and_replies(comment, show_hidden=show_hidden)

The `print_post_and_replies` function recursively prints a comment as well as all replies to that comment, as well as all replies to those replies, etc.

In [6]:
def print_comment_and_replies(comment, num_indents=0, show_hidden=False):
    
    replies = comment.replies

    display_text = (
        comment.body + "\n" +
        "-- " + str(comment.author) + 
        " (score " + str(comment.score) + ")"
    )
    
    if(should_display(comment)):# should_display(tweet_with_replies, tweet_info, replies, author_info, public_metrics)):
        display_indented(display_text, num_indents*20)

        #print replies (and the replies of those, etc.)
        for reply in replies:
            print_comment_and_replies(reply, num_indents = num_indents + 1, show_hidden=show_hidden)
    
    elif(show_hidden): #If we want to still see which posts we are hiding, color them LightCoral so we can see they are hidden
        display_indented(display_text, num_indents*20, color='LightCoral')

## TODO: Create Your Content Moderation Algorithm
Your job is to invent and implement your own rule inside the `should_display` function for what comments count as the "best comments" and therefore should be displayed. The rule can be complicated or simple, it just can't be the same as the current rule. You can aim for focusing on only hiding a few comments that you judge are bad, or for only showing a few comments you judge are the very best, or a combination of those.

When you are making your rule you may want to use different comparison operators (like == for equals, > for greater than, etc.) and different logical operators (like `and` for both things must be true, `or` for at least one thing must be true, etc.). See a list of operators here: https://www.w3schools.com/python/python_operators.asp

Some things you can use when you are deciding whether to display a tweet or not:

* The text of the commnet: `comment.body`
* The score of the comment: `comment.score`
* If the comment is "distinguished": `comment.distinguished`
* If the comment was edited: `comment.edited`
* If the comment was made by the author of the original post: `comment.is_submitter`
* The number of replies: `len(comment.replies)`

You can see more by looking at the official documentation for [lists of attributes of a comment](https://praw.readthedocs.io/en/stable/code_overview/models/comment.html.

You can also look at attributes of the author such as (though you'll have to do an extra `if comment.author` check to make sure the author exists):
* author name: `comment.author.name`
* author comment karma: `comment.author.comment_karma`
* author link karma: `comment.author.link_karma`

You can see more by looking at [lists of attributes of a redditor](https://praw.readthedocs.io/en/stable/code_overview/models/redditor.html#praw.models.Redditor)

* You can use any other information you can figure out about the comment as well, such as the sentiment analysis that was demoed previously.


In [7]:
def should_display(comment):
    #TODO: Make your own rule
    
    # for a demonstration, we only display comments with the lower case letters "the" 
    # Note: that the way we are checking here, a comment that has the word "there" would show up
    #       since "the" appears in "there"
    has_letters_you = "you" in comment.body
    
    if(has_letters_you):
        return True
    else:
        return False

## Finding post IDs and testing our code
In order to test it out, we need to find an id of a reddit post that has comments on it. Once you have a reddit post open in your browser, find or copy the url website address and look for the piece of random letters after `https://www.reddit.com/r/[subredditname]/comments/`, which is the id.

For example, in this post, the id is 'fuulky':
![Screenshot of reddit with a post up. The website url is "https://www.reddit.com//r/MovieDetails/comments/fuulky/in_little_women_2019_laurie_and_jo_swap_articles/". There is a circle drawn around the letters "fuulky" which appears after "comments/"](./imgs/get_reddit_post_id.png)

Now we can test it out by calling the `print_post_and_replies` with the string `'fuulky'` as the argument, and see what comments are displayed.

As you work on your changes to the `should_display` function, you can test it out on different threads by looking up more ids, like: `'vfs5oh'` and `'lzvvwp'`.

In [8]:
print_post_and_replies('13pg774')

Comments and replies for post from /offmychest:


/opt/conda/lib/python3.9/site-packages/praw/models/reddit/submission.py:607: UserWarning: The comments for this submission have already been fetched, so the updated comment_sort will not have any effect
  warn(


If we also want to see what comments are being skipped, we can use an optional argument for `print_post_and_replies` by setting `show_hidden = True`, and the comments that are being skipped will show up with a reddish background.

In [9]:
print_post_and_replies('13pg774', show_hidden = True)

Comments and replies for post from /offmychest:


## TODO! Test it with 3 tweet threads

Now, after you've modified the `should_display`, try testing out your algorithm on three new posts, answering follow up questions after each one.

In the sections below, replace the `?????`s with a reddit post id, and run the code. Then answer the questions about how that went.

At the very end will be more reflection questions.

### TODO: Print reddit thread 1

In [10]:
def should_display(comment):
    has_letters_love = "love" in comment.body
    
    if(has_letters_love):
        return True
    else:
        return False

In [11]:
print_post_and_replies('12wu1fl', show_hidden = True)

Comments and replies for post from /movies:


### TODO: Reddit thread 1 follow-up questions
Write an answer in response to each of these questions (you can edit this text by double clicking it):

Look through the output of `print_post_and_replies()` based on your modified `should_display` function.

Did your function tend to keep most tweets or tend to hide most tweets?

Most of the comments were hidden, but there were a decent amount of comments that contained the word "love".

Do you see any pattern to the contents of the tweets you showed versus hid (e.g., did it actually select better quality or more interesting tweets)?

Most of the red/hidden tweets were shorter which made me believe they were likely comments that were replying to another commenter which explains why the hidden ones were shorter.

### TODO: Print reddit thread 2

In [12]:
def should_display(comment):
    has_letters_hate = "hate" in comment.body
    
    if(has_letters_hate):
        return True
    else:
        return False

In [13]:
print_post_and_replies('13pea6r', show_hidden = True)

Comments and replies for post from /movies:


### TODO: Reddit thread 2 follow-up questions
Write an answer in response to each of these questions (you can edit this text by double clicking it):

Look through the output of `print_post_and_replies()` based on your modified `should_display` function.

Did your function tend to keep most tweets or tend to hide most tweets?

For this function, the majority of the comments were hidden and there were very few comments containing the word "hate".

Do you see any pattern to the contents of the tweets you showed versus hid (e.g., did it actually select better quality or more interesting tweets)?

I was not surprised by the results of this function because I did it intentionally. I searched up a post that asked users their favorite movie and then searched th word "hate" because I was interested to see what the results would be. However, the post is mostly asking about movies that people loved, so I am not surprised that most of the comments are hidden.

### TODO: Print reddit thread 3

In [14]:
def should_display(comment):
    has_letters_happy = "happy" in comment.body
    
    if(has_letters_happy):
        return True
    else:
        return False

In [15]:
print_post_and_replies('ygtckp', show_hidden = True)

Comments and replies for post from /movies:


### TODO: Reddit thread 3 follow-up questions
Write an answer in response to each of these questions (you can edit this text by double clicking it):

Look through the output of `print_post_and_replies()` based on your modified `should_display` function.

Did your function tend to keep most tweets or tend to hide most tweets?

All of the comments were hidden except for one.

Do you see any pattern to the contents of the tweets you showed versus hid (e.g., did it actually select better quality or more interesting tweets)?

Most of them were hidden because I searched for the word "happy" in a post talking about sad movies, just to see if there were any comments about it. So, the results did not surprise me because I searched the antonym for the posts topic.

## TODO: Final Reflection questions
Write an answer in response in response to each of these questions:

Explain why you chose the rules you did for selecting the best comments?

I wanted to find posts containing one word and then create a rule for finding the best comments using a word that is an antonym for the original word. For example, I searched a post asking users about movies they hate and then created a rule searching for comments containing the word "love". Basically, I wanted to find outliers in the comments.

What was most challenging about coming up with your rules?

I would not necessarily say there was anything challenging about it but I think it can be hard to think outside of the box when you are trying to create rules like this. I wanted to do something that was unique and would have an unexpected amount of hidden comments which is why I searched for opposites.

What additional information or rules do you wish you could have used?

I think it would be interesting to use or create a rule that could identify duplicate comments, not just containing a few similar words but comments that are identical to one another, even if they aren't necessarily on the same post. So finding identical comments all throughout reddit.

If someone or some group wanted to make sure their comments were shown by your function, what would they do? How hard would this be?

I think that you could do this by creating a rule that showed all comments connected to that individuals account username. You could use the rule comment.author.name to filter all comments connected to their username. I think this would be relatively easy because it would be able to identify that user and their comments easily, so they would not be hidden.

If someone or some group wanted to make sure someone else's comments were NOT shown by your function, what would they do (if anything)? How hard would this be?

I am not sure how this could be done because there is no way to hide all comments. Maybe you could create a rule that hides all comments created by that user or username. This could likely be done using the comment.author.name function and then adding their username.

If Reddit adopted this rule as a universal rule for which comments to display, what do you think would happen? (e.g., would people change commenting strategies? would comments look different than currently? would it get overwhelmed with spam?)


I think that people would likely change the way they comment in order to get their comments seen more. Whether that is through the language used in their comments or how often or frequently they comment. However, I think it would be hard to create this into a universal rule because once users find out the algorithm and the way around it, they will likely begin flooding the comments with comments in order to get theirs displayed, which would lead to an overwhelming amount of spam comments.